## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-21-21-51-52 +0000,nypost,"Trump tells deployed feds, cops that ‘spotless...",https://nypost.com/2025/08/21/us-news/trump-te...
1,2025-08-21-21-49-04 +0000,nyt,Police Respond to Active Shooter Report at Vil...,https://www.nytimes.com/2025/08/21/us/villanov...
2,2025-08-21-21-46-50 +0000,nypost,Murdered NJ vet Lauren Semanchik left voicemai...,https://nypost.com/2025/08/21/us-news/lauren-s...
3,2025-08-21-21-46-39 +0000,startribune,Live: DFL is revoking Omar Fateh’s endorsement...,https://www.startribune.com/minneapolis-mayor-...
4,2025-08-21-21-45-00 +0000,wsj,Supreme Court Lets Trump Administration Halt F...,https://www.wsj.com/us-news/law/supreme-court-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2327/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,57
97,new,23
106,ukraine,15
214,israel,13
593,russia,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
38,2025-08-21-20-25-04 +0000,startribune,Appeals court throws out massive civil fraud p...,https://www.startribune.com/appeals-court-thro...,120
33,2025-08-21-20-32-00 +0000,wsj,N.Y. Appeals Court Throws Out $500 Million Civ...,https://www.wsj.com/us-news/law/ny-appeals-cou...,116
60,2025-08-21-19-17-34 +0000,nyt,NY Appeals Court Throws Out Trump’s Half-Billi...,https://www.nytimes.com/2025/08/21/nyregion/tr...,115
122,2025-08-21-17-01-27 +0000,nypost,"Trump says he’ll patrol DC with cops, military...",https://nypost.com/2025/08/21/us-news/trump-sa...,110
308,2025-08-21-01-00-00 +0000,wsj,President Trump and his allies are waging a ca...,https://www.wsj.com/economy/central-banking/fe...,109


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
38,120,2025-08-21-20-25-04 +0000,startribune,Appeals court throws out massive civil fraud p...,https://www.startribune.com/appeals-court-thro...
118,63,2025-08-21-17-07-24 +0000,nypost,North Korea’s top officers abruptly pulled out...,https://nypost.com/2025/08/21/world-news/north...
245,54,2025-08-21-09-39-40 +0000,bbc,Palestinians flee Gaza City districts as Israe...,https://www.bbc.com/news/articles/clyr7l0z9edo...
207,52,2025-08-21-12-30-03 +0000,nypost,US keeping lofty EU car tariffs until levies o...,https://nypost.com/2025/08/21/us-news/us-keepi...
44,49,2025-08-21-20-11-18 +0000,nypost,Trump pick Alina Habba is unlawfully serving a...,https://nypost.com/2025/08/21/us-news/trump-pi...
2,47,2025-08-21-21-46-50 +0000,nypost,Murdered NJ vet Lauren Semanchik left voicemai...,https://nypost.com/2025/08/21/us-news/lauren-s...
308,46,2025-08-21-01-00-00 +0000,wsj,President Trump and his allies are waging a ca...,https://www.wsj.com/economy/central-banking/fe...
30,44,2025-08-21-20-39-00 +0000,wsj,A Georgia-based reporter has sued the Trump ad...,https://www.wsj.com/politics/trump-immigration...
0,38,2025-08-21-21-51-52 +0000,nypost,"Trump tells deployed feds, cops that ‘spotless...",https://nypost.com/2025/08/21/us-news/trump-te...
103,36,2025-08-21-17-41-08 +0000,bbc,"Record 111,000 UK asylum applications in past ...",https://www.bbc.com/news/articles/cwy1kxv8xewo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
